In [1]:
import numpy as np
from collections import Counter, defaultdict
import concurrent.futures  # Import for parallel processing

# Define ranks and suits
ranks = np.arange(13)  # Map ranks to integers 0-12 (2 to Ace)
suits = np.arange(4)   # Map suits to integers 0-3 (Hearts, Diamonds, Clubs, Spades)

# Create a deck with two identical sets of 52 cards
deck = np.array([(rank, suit) for rank in ranks for suit in suits] * 2, dtype=[('rank', 'i4'), ('suit', 'i4')])

def print_deck(deck):
    """Print the entire deck to show all cards in both identical sets."""
    for card in deck:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        print(f"{rank_name} of {suit_name}", end=", ")
    print("\n")

def print_hand(hand):
    """Print the hand in a readable 'rank of suit' format."""
    hand_cards = []
    for card in hand:
        rank = card['rank']
        suit = card['suit']
        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][suit]
        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][rank]
        hand_cards.append(f"{rank_name} of {suit_name}")
    print("Hand:", ", ".join(hand_cards))

def deal_hand(deck):
    """
    Shuffle the deck and select five unique cards.
    """
    shuffled_deck = np.random.permutation(deck)
    hand_indices = np.random.choice(len(shuffled_deck), 5, replace=False)
    hand = shuffled_deck[hand_indices]
    sorted_hand = np.sort(hand, order='rank')
    return shuffled_deck, sorted_hand

def check_three_of_a_kind_case(hand):
    rank_counts = Counter([card['rank'] for card in hand])
    three_of_a_kind_rank = None
    other_ranks = set()

    # Identify three-of-a-kind rank and other ranks
    for rank, count in rank_counts.items():
        if count == 3:
            three_of_a_kind_rank = rank
        else:
            other_ranks.add(rank)

    if three_of_a_kind_rank is None or len(other_ranks) != 2:
        return -1  # Not a valid three-of-a-kind as per conditions

    # Collect cards by the three-of-a-kind rank and other ranks
    three_of_a_kind_cards = [card for card in hand if card['rank'] == three_of_a_kind_rank]
    other_cards = [card for card in hand if card['rank'] != three_of_a_kind_rank]

    # Case 0: All three-of-a-kind cards have different suits
    if len(set(card['suit'] for card in three_of_a_kind_cards)) == 3:
        return 0

    # Case 1: Two three-of-a-kind cards are identical in suit, and the third is a different suit
    if len(set(card['suit'] for card in three_of_a_kind_cards)) == 2:
        return 1

    return -1  # Does not match either case
def run_simulation(num_trials, worker_id=None):
    """
    Run simulation with enhanced random seed handling.
    """
    if worker_id is not None:
        base_seed = 104729
        multiplier = 15485863
        worker_seed = base_seed + (worker_id * multiplier)
        np.random.seed(worker_seed)

    counts = Counter()
    local_examples = defaultdict(list)

    batch_size = 100_000
    num_batches = num_trials // batch_size + (1 if num_trials % batch_size else 0)

    for batch in range(num_batches):
        current_batch_size = min(batch_size, num_trials - batch * batch_size)
        batch_counts = Counter()

        for _ in range(current_batch_size):
            _, hand = deal_hand(deck)
            case = check_three_of_a_kind_case(hand)
            if case >= 0:  # Fixed check for valid cases
                batch_counts[case] += 1
                if len(local_examples[case]) < 10:
                    local_examples[case].append(hand.copy())

        counts.update(batch_counts)

        if worker_id is not None:
            print(f"Worker {worker_id}: Completed batch {batch + 1}/{num_batches}, "
                  f"found Case 0: {counts[0]}, "
                  f"Case 1: {counts[1]}, "
                  f"total: {sum(counts.values())} so far")

    return counts, local_examples

def run_simulation_parallel(trials, num_workers=16):
    """
    Run parallel simulation with improved worker management and progress tracking.
    """
    chunk_size = trials // num_workers
    remaining = trials % num_workers
    total_counts = Counter()
    all_examples = defaultdict(list)

    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        # Distribute work with adjusted chunks
        futures = []
        for i in range(num_workers):
            # Last worker gets remaining hands if any
            worker_trials = chunk_size + (remaining if i == num_workers-1 else 0)
            futures.append(executor.submit(run_simulation, worker_trials, worker_id=i))

        # Track progress and collect results
        results = []
        for future in concurrent.futures.as_completed(futures):
            counts, examples = future.result()
            results.append((counts, examples))

    # Combine all results
    for counts, examples in results:
        total_counts.update(counts)
        # Merge examples based on case
        for case in [0, 1]:  # Only Case 0 and Case 1 for three-of-a-kind
            remaining_slots = 10 - len(all_examples[case])
            if remaining_slots > 0 and case in examples:
                all_examples[case].extend(examples[case][:remaining_slots])

    # Print final summary
    print("\nTotal examples collected:")
    for case in [0, 1]:  # Only Case 0 and Case 1
        print(f"Case {case}: {len(all_examples[case])} examples")

    return total_counts, all_examples

def calculate_probability(counts, total_trials):
    """Calculate the probability of three-of-a-kind occurrence."""
    total_count = sum(counts.values())  # Sum all valid cases
    return total_count / total_trials

def main():
    trial_counts = [1_00, 1_000, 10_000, 100_000, 1_000_000, 10_000_000, 100_000_000]
    for trials in trial_counts:
        print(f"\nRunning parallel simulation with {trials} trials...")
        counts, example_hands = run_simulation_parallel(trials)

        print(f"Results for {trials} trials:")
        print(f"Case 0 (all different suits): {counts[0]}")
        print(f"Case 1 (two same suit): {counts[1]}")
        print(f"Total three-of-a-kind hands: {sum(counts.values())}")
        print(f"Probability: {calculate_probability(counts, trials):.8f}")

        # Only print examples for 1 million trials
        if trials == 1_000_000:
            print("\nExample hands from 1,000,000 trials:")

            print("\nCase 0 (all different suits) examples:")
            for i, hand in enumerate(example_hands[0], 1):
                print(f"\nExample {i}:")
                print_hand(hand)
                # Print three-of-a-kind cards specifically
                rank_counts = Counter(card['rank'] for card in hand)
                three_rank = next(rank for rank, count in rank_counts.items() if count == 3)
                print("Three of a kind cards:", end=" ")
                for card in hand:
                    if card['rank'] == three_rank:
                        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][card['suit']]
                        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][card['rank']]
                        print(f"{rank_name} of {suit_name},", end=" ")
                print("\n" + "-"*50)

            print("\nCase 1 (two identical suits) examples:")
            for i, hand in enumerate(example_hands[1], 1):
                print(f"\nExample {i}:")
                print_hand(hand)
                # Print three-of-a-kind cards specifically
                rank_counts = Counter(card['rank'] for card in hand)
                three_rank = next(rank for rank, count in rank_counts.items() if count == 3)
                print("Three of a kind cards:", end=" ")
                for card in hand:
                    if card['rank'] == three_rank:
                        suit_name = ["Hearts", "Diamonds", "Clubs", "Spades"][card['suit']]
                        rank_name = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"][card['rank']]
                        print(f"{rank_name} of {suit_name},", end=" ")
                print("\n" + "-"*50)

if __name__ == "__main__":
    main()





Running parallel simulation with 100 trials...
Worker 0: Completed batch 1/1, found Case 0: 1, Case 1: 0, total: 1 so farWorker 3: Completed batch 1/1, found Case 0: 0, Case 1: 0, total: 0 so farWorker 2: Completed batch 1/1, found Case 0: 0, Case 1: 0, total: 0 so far

Worker 1: Completed batch 1/1, found Case 0: 0, Case 1: 0, total: 0 so farWorker 7: Completed batch 1/1, found Case 0: 0, Case 1: 0, total: 0 so far
Worker 5: Completed batch 1/1, found Case 0: 0, Case 1: 0, total: 0 so far
Worker 4: Completed batch 1/1, found Case 0: 0, Case 1: 0, total: 0 so far
Worker 13: Completed batch 1/1, found Case 0: 0, Case 1: 0, total: 0 so far
Worker 14: Completed batch 1/1, found Case 0: 0, Case 1: 0, total: 0 so farWorker 15: Completed batch 1/1, found Case 0: 0, Case 1: 1, total: 1 so farWorker 8: Completed batch 1/1, found Case 0: 0, Case 1: 0, total: 0 so farWorker 10: Completed batch 1/1, found Case 0: 0, Case 1: 0, total: 0 so far
Worker 12: Completed batch 1/1, found Case 0: 0, Case